In [ ]:
import ml_utils as mlu
import pandas as pd
from sklearn.pipeline import Pipeline
import joblib
import pickle
import numpy as np
from sklearn.metrics import mean_pinball_loss

In [ ]:
train = pd.read_csv("use_to_train.csv")
test = pd.read_csv("use_to_test.csv")
validation = pd.read_csv("use_to_val.csv")

In [ ]:
# Select variables
feature_names_number = ['horsepower', 'displacement', 'torque', 'wheels', 'km', 'age']
feature_names_category = ['navigation_system', 'rear_sensor', 'push_start', 'turbo', 'body_type']

print('feature_names_number')
print(feature_names_number)
print('feature_names_category')
print(feature_names_category)

feature_names_number
['horsepower', 'displacement', 'torque', 'wheels', 'km', 'age']
feature_names_category
['navigation_system', 'rear_sensor', 'push_start', 'turbo', 'body_type']


In [ ]:
X_train = train[feature_names_number + feature_names_category]
y_train = train['price']
X_test = test[feature_names_number + feature_names_category]
y_test = test['price']
X_val = validation[feature_names_number + feature_names_category]
y_val = validation['price']

In [ ]:
# load the saved model
model_pipeline = joblib.load("3_catboost_interval_bcu.joblib")

In [ ]:
# X_train = pd.DataFrame(model_pipeline.fit_transform(X_train))
# X_val = pd.DataFrame(model_pipeline.fit_transform(X_test))
# X_val = pd.DataFrame(model_pipeline.fit_transform(X_val))

In [ ]:
y_train_predict = model_pipeline.predict(X_train)

predictions_train = y_train.to_frame(name="y_true") # the "ground truth" column
predictions_train["pi_median"] = y_train_predict[:, 0]
predictions_train["pi_upper"] = y_train_predict[:, 1]
predictions_train["avg"] = ((predictions_train['pi_median'] + predictions_train['pi_upper'])/2)

train_pinball_50 = pd.DataFrame({'Train_50': [np.round(mean_pinball_loss(predictions_train['y_true'], predictions_train['pi_median'], alpha=0.5),2)]})
train_pinball_75 = pd.DataFrame({'Train_75': [np.round(mean_pinball_loss(predictions_train['y_true'], predictions_train['pi_upper'], alpha=0.75),2)]})

df_train_stats = pd.concat([train_pinball_50, train_pinball_75], axis=1)
df_train_stats

,Train_50,Train_75
0,5198.97,5424.03


In [ ]:
y_test_predict = model_pipeline.predict(X_test)

predictions_test = y_test.to_frame(name="y_true") # the "ground truth" column
predictions_test["pi_median"] = y_test_predict[:, 0]
predictions_test["pi_upper"] = y_test_predict[:, 1]
predictions_test["avg"] = ((predictions_test['pi_median'] + predictions_test['pi_upper'])/2)

test_pinball_50 = pd.DataFrame({'Test_50': [np.round(mean_pinball_loss(predictions_test['y_true'], predictions_test['pi_median'], alpha=0.5),2)]})
test_pinball_75 = pd.DataFrame({'Test_75': [np.round(mean_pinball_loss(predictions_test['y_true'], predictions_test['pi_upper'], alpha=0.75),2)]})

df_test_stats = pd.concat([test_pinball_50, test_pinball_75], axis=1)
df_test_stats

,Test_50,Test_75
0,16667.68,17606.55


In [ ]:
y_val_predict = model_pipeline.predict(X_val)

predictions_val = y_val.to_frame(name="y_true") # the "ground truth" column
predictions_val["pi_median"] = y_val_predict[:, 0]
predictions_val["pi_upper"] = y_val_predict[:, 1]
predictions_val["avg"] = ((predictions_val['pi_median'] + predictions_val['pi_upper'])/2)

val_pinball_50 = pd.DataFrame({'Val_50': [np.round(mean_pinball_loss(predictions_val['y_true'], predictions_val['pi_median'], alpha=0.5),2)]})
val_pinball_75 = pd.DataFrame({'Val_75': [np.round(mean_pinball_loss(predictions_val['y_true'], predictions_val['pi_upper'], alpha=0.75),2)]})

df_val_stats = pd.concat([val_pinball_50, val_pinball_75], axis=1)
df_val_stats

,Val_50,Val_75
0,15341.96,14426.32


In [ ]:
full_stats = pd.concat([df_train_stats, df_test_stats, df_val_stats], axis=1)
full_stats

,Train_50,Train_75,Test_50,Test_75,Val_50,Val_75
0,5198.97,5424.03,16667.68,17606.55,15341.96,14426.32


In [ ]:
def pinball_loss(y_true, y_pred, alpha=0.5):
    delta = y_true - y_pred
    return np.where(delta > 0, alpha * delta, (alpha - 1) * delta)

In [ ]:
preds_train = pd.DataFrame(y_train_predict)
preds_train.columns = ['y_50', 'y_75']
df_prices_train = pd.concat([y_train, preds_train], axis=1)
df_prices_train['Quantil_loss_50'] = df_prices_train.apply(
    lambda row: np.round(pinball_loss(row['price'], row['y_50'], alpha=0.5), 2), axis=1
)
df_prices_train['Quantil_loss_75'] = df_prices_train.apply(
    lambda row: np.round(pinball_loss(row['price'], row['y_75'], alpha=0.75), 2), axis=1
)
df_prices_train.head()

,price,y_50,y_75,Quantil_loss_50,Quantil_loss_75
0,451999,453621.217289,465998.332332,811.11,3499.83
1,281999,283515.207976,276730.783842,758.10,3951.16
2,224999,224456.056119,226504.041646,271.47,376.26
3,171999,189828.001832,191933.105431,8914.50,4983.53
4,199999,195865.932351,199880.709152,2066.53,88.72


In [ ]:
display(df_prices_train['Quantil_loss_50'].mean(), df_prices_train['Quantil_loss_75'].mean())

5198.96583778015

5424.029220917822

In [ ]:
preds_test = pd.DataFrame(y_test_predict)
preds_test.columns = ['y_50', 'y_75']
df_prices_test = pd.concat([y_test, preds_test], axis=1)
df_prices_test['Quantil_loss_50'] = df_prices_test.apply(
    lambda row: np.round(pinball_loss(row['price'], row['y_50'], alpha=0.5), 2), axis=1
)
df_prices_test['Quantil_loss_75'] = df_prices_test.apply(
    lambda row: np.round(pinball_loss(row['price'], row['y_75'], alpha=0.75), 2), axis=1
)
df_prices_test.head()

,price,y_50,y_75,Quantil_loss_50,Quantil_loss_75
0,206999,205502.931323,207772.618183,748.03,193.40
1,192999,200247.037322,201756.362803,3624.02,2189.34
2,791999,655809.225519,663665.561848,68094.89,96250.08
3,418999,425048.640981,427288.264748,3024.82,2072.32
4,210999,220130.014352,219665.510361,4565.51,2166.63


In [ ]:
display(df_prices_test['Quantil_loss_50'].mean(), df_prices_test['Quantil_loss_75'].mean())

16667.682136752137

17606.546004273503

In [ ]:
preds_val = pd.DataFrame(y_val_predict)
preds_val.columns = ['y_50', 'y_75']
df_prices_val = pd.concat([y_val, preds_val], axis=1)
df_prices_val['Quantil_loss_50'] = df_prices_val.apply(
    lambda row: np.round(pinball_loss(row['price'], row['y_50'], alpha=0.5), 2), axis=1
)
df_prices_val['Quantil_loss_75'] = df_prices_val.apply(
    lambda row: np.round(pinball_loss(row['price'], row['y_75'], alpha=0.75), 2), axis=1
)
df_prices_val.head()

,price,y_50,y_75,Quantil_loss_50,Quantil_loss_75
0,172999,160204.944346,178236.183021,6397.03,1309.30
1,181999,181626.267328,178698.170190,186.37,2475.62
2,390999,347537.042729,318981.631790,21730.98,54013.03
3,620999,674746.565211,723735.197100,26873.78,25684.05
4,229999,237868.719639,245485.110722,3934.86,3871.53


In [ ]:
display(df_prices_val['Quantil_loss_50'].mean(), df_prices_val['Quantil_loss_75'].mean())

15341.955607675907

14426.319104477612

In [ ]:
interval_range = 50000
min_price = 100000
max_price = 1000000


In [ ]:
def get_df_summary(min_price, max_price, interval_range):
    intervals_start = np.arange(min_price, max_price, interval_range)
    df_start = pd.DataFrame(intervals_start)
    df_start.columns = ['Starting']
    intervals_end = np.arange(min_price + interval_range, max_price + interval_range, interval_range)
    df_end = pd.DataFrame(intervals_end)
    df_end.columns = ['Ending']
    df_summary = pd.concat([df_start, df_end], axis=1)
    return df_summary

In [ ]:
def get_count(row, df_values, price_column, sum_column):
    a = row['Starting']
    b = row['Ending']
    # Filtrar los valores entre a y b
    valores_filtrados = df_values[(df_values[price_column] >= a) & (df_values[price_column] <= b)]
    # Calcular el promedio
    count = np.round(valores_filtrados[sum_column].count(), 2)
    return count

In [ ]:
def get_avg_error(row, df_values, price_column, sum_column):
    a = row['Starting']
    b = row['Ending']
    # Filtrar los valores entre a y b
    valores_filtrados = df_values[(df_values[price_column] >= a) & (df_values[price_column] <= b)]
    # Calcular el promedio
    promedio = np.round(valores_filtrados[sum_column].mean(), 2)
    return promedio

In [ ]:
df_sum = get_df_summary(min_price, max_price, interval_range)
df_sum['Qloss_mean_50'] = df_sum.apply(lambda row: get_avg_error(row, df_prices_train, 'price', 'Quantil_loss_50'), axis=1)
df_sum['Qloss_mean_75'] = df_sum.apply(lambda row: get_avg_error(row, df_prices_train, 'price', 'Quantil_loss_75'), axis=1)
df_sum['Count'] = df_sum.apply(lambda row: get_count(row, df_prices_train, 'price', 'Quantil_loss_75'), axis=1)
df_sum

,Starting,Ending,Qloss_mean_50,Qloss_mean_75,Count
0,100000,150000,4764.08,7500.91,9
1,150000,200000,3345.99,3705.48,146
2,200000,250000,4734.22,4553.00,199
3,250000,300000,4403.03,5089.79,168
4,300000,350000,5892.19,7062.59,136
5,350000,400000,6869.15,6665.22,103
6,400000,450000,7331.97,6584.73,52
7,450000,500000,5461.09,4940.29,47
8,500000,550000,6434.65,4478.71,29
9,550000,600000,3000.40,4019.62,20


In [ ]:
df_sum = get_df_summary(min_price, max_price, interval_range)
df_sum['Qloss_mean_50'] = df_sum.apply(lambda row: get_avg_error(row, df_prices_test, 'price', 'Quantil_loss_50'), axis=1)
df_sum['Qloss_mean_75'] = df_sum.apply(lambda row: get_avg_error(row, df_prices_test, 'price', 'Quantil_loss_75'), axis=1)
df_sum['Count'] = df_sum.apply(lambda row: get_count(row, df_prices_test, 'price', 'Quantil_loss_75'), axis=1)
df_sum

,Starting,Ending,Qloss_mean_50,Qloss_mean_75,Count
0,100000,150000,8239.13,6801.28,9
1,150000,200000,5750.86,5035.88,62
2,200000,250000,9049.18,8608.84,93
3,250000,300000,13734.46,12759.63,97
4,300000,350000,17278.28,14032.08,69
5,350000,400000,14723.98,17092.78,36
6,400000,450000,25518.23,24998.35,39
7,450000,500000,27850.07,26036.92,17
8,500000,550000,27039.75,25125.79,13
9,550000,600000,30579.36,34296.13,10


In [ ]:
df_sum[['Qloss_mean_50', 'Qloss_mean_75']][:10].mean()

Qloss_mean_50    17976.330
Qloss_mean_75    17478.768
dtype: float64

In [ ]:
df_sum[:10]

,Starting,Ending,Qloss_mean_50,Qloss_mean_75,Count
0,100000,150000,8239.13,6801.28,9
1,150000,200000,5750.86,5035.88,62
2,200000,250000,9049.18,8608.84,93
3,250000,300000,13734.46,12759.63,97
4,300000,350000,17278.28,14032.08,69
5,350000,400000,14723.98,17092.78,36
6,400000,450000,25518.23,24998.35,39
7,450000,500000,27850.07,26036.92,17
8,500000,550000,27039.75,25125.79,13
9,550000,600000,30579.36,34296.13,10


In [ ]:
df_sum[['Qloss_mean_50', 'Qloss_mean_75']][:10].mean()

Qloss_mean_50    17976.330
Qloss_mean_75    17478.768
dtype: float64

In [ ]:
df_sum = get_df_summary(min_price, max_price, interval_range)
df_sum['Qloss_mean_50'] = df_sum.apply(lambda row: get_avg_error(row, df_prices_train, 'price', 'Quantil_loss_50'), axis=1)
df_sum['Qloss_mean_75'] = df_sum.apply(lambda row: get_avg_error(row, df_prices_train, 'price', 'Quantil_loss_75'), axis=1)
df_sum['Count'] = df_sum.apply(lambda row: get_count(row, df_prices_train, 'price', 'Quantil_loss_75'), axis=1)
df_sum

,Starting,Ending,Qloss_mean_50,Qloss_mean_75,Count
0,100000,150000,4764.08,7500.91,9
1,150000,200000,3345.99,3705.48,146
2,200000,250000,4734.22,4553.00,199
3,250000,300000,4403.03,5089.79,168
4,300000,350000,5892.19,7062.59,136
5,350000,400000,6869.15,6665.22,103
6,400000,450000,7331.97,6584.73,52
7,450000,500000,5461.09,4940.29,47
8,500000,550000,6434.65,4478.71,29
9,550000,600000,3000.40,4019.62,20


In [ ]:
df_sum[['Qloss_mean_50', 'Qloss_mean_75']][:10].mean()

Qloss_mean_50    5223.677
Qloss_mean_75    5460.034
dtype: float64

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a817877f-bc24-4404-a1bc-b1f4f599a592' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>